In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [2]:
train_data = catalog.load('train.events_data')
labels_q_train = catalog.load('train.labels_q')
lk_question = catalog.load('lk_question')

[05/09/23 22:15:58] INFO     Loading data from 'train.events_data' (PickleDataSet)...           ]8;id=696470;file:///Users/iferreiro/miniconda3/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=910682;file:///Users/iferreiro/miniconda3/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'train.labels_q' (PickleDataSet)...              ]8;id=210398;file:///Users/iferreiro/miniconda3/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=996287;file:///Users/iferreiro/miniconda3/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'lk_question' (PickleDataSet)...                 ]8;id=253399;file:///Users/iferreiro/miniconda3/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=161880;file:///Users/iferreiro/miniconda3/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [3]:
display(train_data.head())

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,level_group,time_diffs
881,20090312433251036,0,0,cutscene_click,basic,0,NaN,-395.0,84.6875,399.0,250.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0-4,0.0
882,20090312433251036,1,218,person_click,basic,0,NaN,-395.0,84.6875,399.0,250.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,218.0
883,20090312433251036,2,448,person_click,basic,0,NaN,-395.0,84.6875,399.0,250.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,230.0
884,20090312433251036,3,702,person_click,basic,0,NaN,-395.0,84.6875,399.0,250.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,254.0
885,20090312433251036,4,948,person_click,basic,0,NaN,-395.0,84.6875,399.0,250.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,246.0


In [12]:
display(lk_question.head(5))
level_groups = list(lk_question['level_group'].unique())

,level_group
1,0-4
2,0-4
3,0-4
4,5-12
5,5-12


# Feature engineering
## Clicks with text

In [5]:
text_pvt = []
i = 0
for lvl_grp in level_groups:
    text_pvt.append(pd.pivot_table(train_data[(train_data['level_group'] == lvl_grp)
                                                  & (train_data['event_name'].isin(['person_click',
                                                                                   'cutscene_click',
                                                                                   'observation_click',
                                                                                   'notification_click']))],
                                index=['session_id', 'level_group'],
                                columns=['event_name', 'text_fqid'],
                                aggfunc={
                                    'elapsed_time': ['min', 'count'],
                                    'text': 'nunique',
                                    'level': 'min'
                                },
                               fill_value=0,
                               observed = True,
                               dropna = True))
    unique_values_per_column = text_pvt[i].nunique()
    cols_to_drop = unique_values_per_column[unique_values_per_column == 1].index
    text_pvt[i].drop(cols_to_drop, axis=1, inplace = True)
    i += 1

In [6]:
for i in range (0,3):
    display(text_pvt[i].head())

elapsed_time   
                                                            count   
event_name                                         cutscene_click   
text_fqid                     tunic.capitol_0.hall.chap1_finale_c   
session_id        level_group                                       
20090312433251036 0-4                                           0  \
20090312455206810 0-4                                           0   
20090313091715820 0-4                                           0   
20090314035813970 0-4                                           0   
20090314121766812 0-4                                           0   

                                                                     
                                                                     
event_name                                                           
text_fqid                     tunic.historicalsociety.closet.intro   
session_id        level_group                                        
20090312433251036 0-4                                            4  \
20090312455206810 0-4                                            8   
20090313091715820 0-4                                            3   
20090314035813970 0-4                                            1   
20090314121766812 0-4                                            1   

                                                                      
                                                                      
event_name                                                            
text_fqid                     tunic.historicalsociety.collection.cs   
session_id        level_group                                         
20090312433251036 0-4                                             8  \
20090312455206810 0-4                                            11   
20090313091715820 0-4                                             8   
20090314035813970 0-4                                            13   
20090314121766812 0-4                                             8   

                                                                         
                                                                         
event_name                                                               
text_fqid                     tunic.historicalsociety.entry.groupconvo   
session_id        level_group                                            
20090312433251036 0-4                                               22  \
20090312455206810 0-4                                               28   
20090313091715820 0-4                                               21   
20090314035813970 0-4                                               15   
20090314121766812 0-4                                               15   

                                                                     
                                                                     
event_name                                                           
text_fqid                     tunic.kohlcenter.halloffame.togrampa   
session_id        level_group                                        
20090312433251036 0-4                                            2  \
20090312455206810 0-4                                            2   
20090313091715820 0-4                                            2   
20090314035813970 0-4                                            2   
20090314121766812 0-4                                            2   

                                                                        
                                                                        
event_name                                         notification_click   
text_fqid                     tunic.historicalsociety.closet.notebook   
session_id        level_group                                           
20090312433251036 0-4                                               1  \
20090312455206810 0-4                                               1 

elapsed_time   
                                                            count   
event_name                                         cutscene_click   
text_fqid                     tunic.capitol_1.hall.chap2_finale_c   
session_id        level_group                                       
20090312433251036 5-12                                          0  \
20090312455206810 5-12                                          0   
20090313091715820 5-12                                          1   
20090314035813970 5-12                                          0   
20090314121766812 5-12                                          0   

                                                                                      
                                                                                      
event_name                                                                            
text_fqid                     tunic.historicalsociety.closet_dirty.door_block_clean   
session_id        level_group                                                         
20090312433251036 5-12                                                         0     \
20090312455206810 5-12                                                         0      
20090313091715820 5-12                                                         0      
20090314035813970 5-12                                                         0      
20090314121766812 5-12                                                         2      

                                                                                     
                                                                                     
event_name                                                                           
text_fqid                     tunic.historicalsociety.closet_dirty.door_block_talk   
session_id        level_group                                                        
20090312433251036 5-12                                                         0    \
20090312455206810 5-12                                                         0     
20090313091715820 5-12                                                         0     
20090314035813970 5-12                                                         0     
20090314121766812 5-12                                                         1     

                                                                                    
                                                                                    
event_name                                                                          
text_fqid                     tunic.historicalsociety.closet_dirty.trigger_coffee   
session_id        level_group                                                       
20090312433251036 5-12                                                         2   \
20090312455206810 5-12                                                         3    
20090313091715820 5-12                                                         2    
20090314035813970 5-12                                                         2    
20090314121766812 5-12                                                         2    

                                                                                   
                                                                                   
event_name                                                                         
text_fqid                     tunic.historicalsociety.closet_dirty.trigger_scarf   
session_id        level_group                                                      
20090312433251036 5-12                                                         5  \
20090312455206810 5-12                                                         5   
20090313091715820 5-12                                                         5   
20090314035813970 5-12                                                         5   
20090314121766812 5-12 

elapsed_time   
                                                            count   
event_name                                         cutscene_click   
text_fqid                     tunic.capitol_2.hall.chap4_finale_c   
session_id        level_group                                       
20090312433251036 13-22                                         0  \
20090312455206810 13-22                                         0   
20090313091715820 13-22                                         0   
20090314035813970 13-22                                         1   
20090314121766812 13-22                                         0   

                                                                          
                                                                          
event_name                                                                
text_fqid                     tunic.historicalsociety.basement.ch3start   
session_id        level_group                                             
20090312433251036 13-22                                               9  \
20090312455206810 13-22                                              10   
20090313091715820 13-22                                              11   
20090314035813970 13-22                                               8   
20090314121766812 13-22                                               8   

                                                                            
                                                                            
event_name                                                                  
text_fqid                     tunic.historicalsociety.basement.savedteddy   
session_id        level_group                                               
20090312433251036 13-22                                                13  \
20090312455206810 13-22                                                 6   
20090313091715820 13-22                                                11   
20090314035813970 13-22                                                11   
20090314121766812 13-22                                                 7   

                                                                              
                                                                              
event_name                                                                    
text_fqid                     tunic.historicalsociety.basement.seescratches   
session_id        level_group                                                 
20090312433251036 13-22                                                  11  \
20090312455206810 13-22                                                  11   
20090313091715820 13-22                                                  11   
20090314035813970 13-22                                                  11   
20090314121766812 13-22                                                  11   

                                                                           
                                                                           
event_name                                                                 
text_fqid                     tunic.historicalsociety.cage.confrontation   
session_id        level_group                                              
20090312433251036 13-22                                               20  \
20090312455206810 13-22                                               19   
20090313091715820 13-22                                               21   
20090314035813970 13-22                                               35   
20090314121766812 13-22                                               20   

                                                                        
                                                                        
event_name                                                              
text_fqid                     tunic.historicalsociety.cage.unlockdoor

# Model training

In [27]:
clf = []
model_metadata = []
gbm_param_grid = {
    'colsample_bytree': [0.3, 0.5],
    'n_estimators': [100],
    'learning_rate': [0.3, 0.5],
    'max_depth': [4, 6]
}

for q in range(0,len(lk_question)):
    lvl_grp_pos = int(np.where(lk_question.loc[lk_question.index.values[q],:] == '0-4', 0,
                           np.where(lk_question.loc[lk_question.index.values[q],:] == '5-12', 1, 2)))
    X = text_pvt[lvl_grp_pos].sort_index()
    y = labels_q_train[q].sort_index()
    assert list(X.index.get_level_values('session_id')) == list(y.index.values)
    X, y = (X.to_numpy(), y.to_numpy())
    dtrain_reg = xgb.DMatrix(X, y)
    clf_q = xgb.XGBClassifier()
    grid_search = GridSearchCV(estimator = clf_q, param_grid = gbm_param_grid,
                               cv = 3, scoring = "roc_auc", verbose = 1)
    grid_search.fit(X, y)
    model_metadata.append({"best_score": grid_search.best_score_,
                   "best_params": grid_search.best_params_})
    clf.append(grid_search.best_estimator_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 8 candidates

In [29]:
for i in range(0, len(model_metadata)):
    print (model_metadata[i]['best_score'])

0.6370008909074286
0.6136554552526635
0.6140030595929846
0.6847576041270824
0.6391309911048394
0.6599903161022079
0.6171163803555593
0.5587623255895392
0.6352838378461851
0.624597177802912
0.5863243758328617
0.6027597048948398
0.6218593713478275
0.6582398117051041
0.6681268546984659
0.5333346854111968
0.559220770402214
0.6717711103881321


In [38]:
import pickle
with open('xgb_clf', 'wb') as f:
    pickle.dump(clf, f)

In [40]:
with open('xgb_clf', 'rb') as f:
    clf_copy = pickle.load(f)
    

In [41]:
print(clf_copy[0])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)
